# Introduction to Movements

Install the mahotas library that has the Zernike moments function we will be using.

In this exercise, we will be analyzing the Hu and Zernike moments of images. The images are a set of 6 classes of plankton, with four images from each class, captured by an underwater microscope (Imaging FlowCytobot), provided by the Woods Hole Open Access Server (https://darchive.mblwhoilibrary.org/handle/1912/7341). Here are some web sites and videos to provide an introduction to moments.  

1. TED talk by Fei Fei Li (Stanford) on computers understanding pictures https://youtu.be/40riCqvRoMs?list=PLNL0bbRjhNhDmIzUoIfijD5s4T9vreP9Q  
2. Geometric properties of binary images https://youtu.be/ZPQiKXqHYrM
3. Intro to image moments https://www.youtube.com/watch?v=AAbUfZD_09s 
4. Mathematical basis of moments https://youtu.be/ISaVvSO_3Sg 
5. Hu moments https://youtu.be/uEVrJrJfa0s?list=PLNL0bbRjhNhDmIzUoIfijD5s4T9vreP9Q 
6. Visualizing Zernike moments https://youtu.be/ESr3Uiqt4xs 

# Setting up the Environment

In [ ]:
!pip install mahotas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.7 MB 4.4 MB/s 
ERROR: Operation cancelled by user


Load the drive with the images of 6 classes of plankton, with 4 sammples, for a total of 24 images.

In [ ]:
drive.mount('/content/drive')
DIR = r'/content/drive/MyDrive/SCIP_DATA/Images/MomentImages'

NameError: ignored

Load all the libraries we will need.

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from mahotas.zernike import zernike_moments
from google.colab.patches import cv2_imshow
from IPython import display
from google.colab import drive


Define two functions that receive a binary image and return 7 Hu moments and 25 Zernike moments, respectively. We will be calling these functions later in the code. The Hu moments have a wide range of values. To make them easier to view when plotting, we will take the log values, remove the sign, then scale it by a factor of 1000 and just use the integer value.

# Create Hu and Zernike Functions

In [ ]:
def getHu(binaryIM):
    mom = cv2.HuMoments(cv2.moments(binaryIM)).flatten()
    momLog=abs(np.log10(np.abs(mom))) # get absolute value of log
    momLog=(1000*momLog).astype(int)
    return(momLog)

def getZernike(binaryIM):
    # ZERNIKE MOMENTS
    W, H = binaryIM.shape
    R = min(W, H) / 2
    z = zernike_moments(binaryIM, R, 8)
    zsum=np.sum(z[1:])
    z[0]=zsum           # first Zernike moment always constant so replace with sum
    return(z)

# View Image Dataset

Let's first look at the images we will be analyzing. Notice the file naming convention is A_B.jpg where A is the planton class (species), and B is a sample image from the class. This will be important to remember when we plot moments for all the images. 

In [ ]:
files = [entry for entry in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, entry))]

for file in files:
  print(file)
  grayIM = cv2.imread(DIR+'/'+file,0) # load image as grayscale
  cv2_imshow(grayIM)


# Calculate Moments

Now we are ready to calculate the Hu and Zernike moments. We will store them in numpy arrays so we can access them later many times without re-calculating the values. In order to calculate the moments, the functions need a binary image (only black or white, 0 or 255, no inbetween grey values). We use the threshold function to convert the grayscale image into a binary image. Notice also we use the parameter cv2.THRESH_BINARY_INV which inverts the binary image, since the function wants a white object on a black background, while our original image is a black object on a white background.

There is enough variation in the image brightness values among the collection of images, that it is difficult to set one threshold that is adequate for all the images. So instead, for each image we will take the average brightness (mean), and set the threshould slightly below (80% of the man) the average, to distinguish the dark object from the bright background. 

In [ ]:
Z_TERMS=25
H_TERMS=7
files = [entry for entry in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, entry))]
hh=np.zeros((len(files),H_TERMS))
zz=np.zeros((len(files),Z_TERMS))
i=0
fileName=[]
for file in files:
    grayIM = cv2.imread(DIR+'/'+file,0) # load image as grayscale
    gm=grayIM.mean()  # get the average brightness of the image
    th=int(gm*0.8)    # set the threshold slightly lower than the average brightness
    ret,binaryIM = cv2.threshold(grayIM,th,255,cv2.THRESH_BINARY_INV) # threshold image, take inverse to make white object on black background
    hh[i] = getHu(binaryIM)
    zz[i]=getZernike(binaryIM)
    fileName.append(file)
    i+=1    


# Plot Moments

Now we are ready to look at the results. There are 24 images, 7 Hu moments, 25 Zernike moments. How can we plot all these items? We will make two plots, one for the Hu and one for the Zernike moments. Each plot will look at only one moment term. To distinguish the species and samples (members), we will color code the species. 

**TO DO AND NOTICE:**
Change the zTerm value and re-run the code. In a later class, we will be using moments to classify species. Ideally the moment value for images of the same species should be close together (intra-species values), and images of different species should be far apart (inter-species values). 

Let's start with the Zernike moments....

In [ ]:
zTerm=0 #0-24
color=['r','g','b','c','m','y'] # species 0 = red, species 1 = green, etc
for species in range(6):  
    for member in range(4):
        imageNumber=species*4+member
        value=zz[imageNumber,zTerm]
        plt.scatter(imageNumber,value,marker='o',color=color[species])    
name='Zernike Term=' + str(zTerm)   # create a title for the plot
plt.title(name)
plt.show() 

Now lets examine the Hu Moments.

**TO DO AND NOTICE:** Change the hTerm value and re-run the code. 

In [ ]:
hTerm=2 #0-6
color=['r','g','b','c','m','y']
for species in range(6):
    for member in range(4):
        imageNumber=species*4+member
        value=hh[imageNumber,hTerm]
        plt.scatter(imageNumber,value,marker='o',color=color[species])  
name='Hu Term=' + str(hTerm)
plt.title(name)
plt.show() 